In [29]:
stanza.download("fr")
stanza.download("nl")

2021-03-26 14:56:52 INFO: Downloading default packages for language: fr (French)...
2021-03-26 15:06:06 INFO: Finished downloading models and saved to /home/hanh/stanza_resources.
2021-03-26 15:06:07 INFO: Downloading default packages for language: nl (Dutch)...
2021-03-26 15:16:25 INFO: Finished downloading models and saved to /home/hanh/stanza_resources.


In [1]:
from collections import Counter
import pandas as pd
import stanza
import re

In [54]:
en_gt = pd.read_csv('../ACTER/nl/htfl/annotations/htfl_nl_terms_nes.ann', sep='	', engine='python',header=None)
en_preds = pd.read_csv('../results/nl/nes/bert_multilingual_uncased_nl_nes_predictions.csv')
preds = list(en_preds['predictions'])
gts = list(en_gt[0])

In [55]:
def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

In [56]:
preds_len = [len(x.split(' ')) for x in preds]
gts_len = [len(x.split(' ')) for x in gts]
print(Counter(preds_len))
print(Counter(gts_len))

Counter({1: 1314, 2: 41, 3: 11, 4: 1})
Counter({1: 1687, 2: 391, 3: 108, 4: 35, 5: 14, 6: 6, 8: 3, 11: 2, 9: 2, 7: 2, 10: 2, 14: 1, 12: 1})


In [57]:
max(gts, key=len)

'european trial on reduction of cardiac events with perindopril in stable coronary artery disease'

In [58]:
# P = TP/(TP+FP)
# R =  TP/(TP+FN)
def evaluation_metrics(pred, gt):
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP, FP, FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score

precision, recall, f1_score = evaluation_metrics(preds, gts)
print(len(set(preds)))
print(len(set(gts)))
precision, recall, f1_score

932 435 1322
1367
2254


(68.18, 41.35, 51.48)

Stanza

In [59]:
nlp = stanza.Pipeline(lang='nl')# processors='tokenize,mwt,pos,lemma'

2021-03-26 16:13:23 INFO: Loading these models for language: nl (Dutch):
| Processor | Package |
-----------------------
| tokenize  | alpino  |
| pos       | alpino  |
| lemma     | alpino  |
| depparse  | alpino  |
| ner       | conll02 |

2021-03-26 16:13:23 INFO: Use device: cpu
2021-03-26 16:13:23 INFO: Loading: tokenize
2021-03-26 16:13:23 INFO: Loading: pos
2021-03-26 16:13:24 INFO: Loading: lemma
2021-03-26 16:13:24 INFO: Loading: depparse
2021-03-26 16:13:24 INFO: Loading: ner
2021-03-26 16:13:27 INFO: Done loading processors!


In [60]:
def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

In [61]:
preds_lem = lemma(preds)
gts_lem = lemma(gts)

print('After lemmatization')
print(len(set(preds_lem)))
print(len(set((gts_lem))))
print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

After lemmatization
1310
2088
892 418 1196
(68.09, 42.72, 52.5)


In [62]:
set(preds_lem)

{'hartfaal',
 'hormoon',
 'hartcyclus',
 'alat',
 'eccles',
 'systolisch',
 'factoranalyse',
 'basisreanimatie',
 'gastro enteritis',
 'calciumantagonist',
 'angiotensine converzymremmer',
 'creatinine',
 'perindoprilgebruik',
 'voorwaarts',
 'eerstelijnsgezondheidszorg',
 'pulmonaal',
 'cox 2 remmer',
 'geneesmiddelvoorschrift',
 'training',
 'streefdosering',
 'sexual adjustment scale',
 'negroïde',
 'galopritme',
 'nierfunctieverslechtering',
 'slagvolume',
 'fret',
 'omapatrilaat',
 'dehydratie',
 'hartminuutvolume',
 'cardiac event',
 'serumcreatinine',
 'cavum',
 'spirometrie',
 'vaatziekte',
 'hartfalenpoli',
 'doel',
 'spanje',
 'cardiothoracaal ratio',
 'hartspiercelverval',
 'prostaathypertrofie',
 'carvedilolgroep',
 'ventrikel .1',
 'beleid',
 'relaxatietijd',
 'rosiglitazon',
 'aangrijpingspunt',
 'defibrilleren',
 'ooghoogte',
 'torasemide',
 'chirurgisch',
 'furosemide',
 'digoxine spiegel',
 'thorax',
 'overbelasting',
 'medicament',
 'epd',
 'kaliumsparend',
 'plasmaco

In [63]:
set(gts_lem)

{'allel',
 'athene',
 'hartfaal',
 "rct'",
 'american thoracic society',
 'hormoon',
 'cardioselectief bhtablokker',
 'alat',
 'hartcyclus',
 'eccles',
 'systolisch',
 'cardiovasculaire ziekte',
 'antagoneren',
 'factoranalyse',
 'positief venepols',
 'basisreanimatie',
 't2 receptoor',
 'dyspnée',
 'gastro enteritis',
 'angiotensine converzymremmer',
 'calciumantagonist',
 'creatinine',
 'vesiculair ademgeruis',
 'aldosteronproductie',
 'perindoprilgebruik',
 'eerstelijnsgezondheidszorg',
 'pulmonaal',
 'cox 2 remmer',
 'geneesmiddelvoorschrift',
 'glycine',
 'plaatjesaggregatie',
 'sexual adjustment scale',
 'thymidinefosforylase',
 'galopritme',
 'neurohumoraal systeem',
 'nierfunctieverslechtering',
 'slagvolume',
 'reversibiliteit',
 'cardiovasculaire aandoening',
 'specificiteit',
 'embolisch',
 'charm alternative',
 'perfusie',
 'fret',
 'omapatrilaat',
 'dehydratie',
 'hartminuutvolume',
 'serumcreatinine',
 'tachyaritmieën',
 'spirometrie',
 'vaatziekte',
 'hartfalenpoli',
 'e

NLTK

In [13]:
# from nltk.stem import PorterStemmer, WordNetLemmatizer
# port = PorterStemmer()
# wnl = WordNetLemmatizer()

# preds_stem = [port.stem(i) for i in preds]
# gts_stem = [port.stem(i) for i in gts]

# print('After stemming')
# print(len(set(preds_stem)))
# print(len(set(gts_stem)))
# print(evaluation_metrics(list(set(preds_stem)), list(set(gts_stem))))

# preds_lem = [wnl.lemmatize(i) for i in preds]
# gts_lem = [wnl.lemmatize(i) for i in gts]

# print('After lemmatization')
# print(len(set(preds_lem)))
# print(len(set((gts_lem))))
# print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

# preds_com = [port.stem(wnl.lemmatize(i)) for i in preds]
# gts_com = [port.stem(wnl.lemmatize(i)) for i in gts]

# print('After combined')
# print(len(set(preds_com)))
# print(len(set((gts_com))))
# evaluation_metrics(list(set(preds_com)), list(set(gts_com)))

# preds_ =[x for x in preds if len(x) >= 2]
# gts_ =[x for x in gts if len(x) >= 2]
# print(len(set(preds_)))
# print(len(gts))
# evaluation_metrics(set(preds_), gts)